# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
# E-commerce customer service chatbot
# Has 4 tools
# 1. Gets the order id given a user-name and item name
# 2. Gets the item status given an order id
# 3. If the order delivery date for orders that are in In Transit

In [53]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr
import json

In [42]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key not set


In [4]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()

claude = anthropic.Anthropic()

In [107]:
# A generic system message - no more snarky adversarial AIs!

system_message = """You are a helpful customer support executive working in a e-commerce store, 
you are responsible to resolve user queries related to users receiving their orders. 
When a tool response has 'partial_result', determine if another tool call is necessary.
If another tool call is needed, make the call IMMEDIATELY without waiting for user input.
Only respond to the user when all required tool calls are completed."""

In [96]:
# 1. Gets the order id given a user-name and item name

nameToOrderIdMap = {"karishma": { "January 30, 2024": "order1", "January 25, 2024": "order2" }, "dinesh": { "January 20, 2024": "order2"} }

def get_order_for(user_name, order_date):
    print(f"Tool get_order_for called for user: {user_name}, date: {order_date}")
    user_name_lower = user_name.lower()
    user_orders = nameToOrderIdMap.get(user_name_lower)
    if user_orders:
        return user_orders.get(order_date)

In [97]:
print(get_order_for("karishma", "January 25, 2024"))
print(get_order_for("karishma", "January 26, 2024"))
print(get_order_for("dishu", "January 26, 2024"))

Tool get_order_for called for user: karishma, date: January 25, 2024
order2
Tool get_order_for called for user: karishma, date: January 26, 2024
None
Tool get_order_for called for user: dishu, date: January 26, 2024
None


In [98]:
# 2. Gets the item status given an order id
orderIdToStatusMap = {"order1": "In Transit" , "order2": "Delivered on February 28, 2024", "order3": "Ready to ship" }

def get_order_status(order_id):
    print(f"Tool get_order_status called for orderId: {order_id}")
    order_id_lower = order_id.lower()
    return orderIdToStatusMap.get(order_id_lower, "Unknown")

In [99]:
print(get_order_status("order1"))
print(get_order_status("order5"))

Tool get_order_status called for orderId: order1
In Transit
Tool get_order_status called for orderId: order5
Unknown


In [100]:
# 3. If the order delivery date for orders that are in In Transit
orderIdToDeliveryDateMap = {"order1": "February 16, 2024" }

def get_order_delivery_date(order_id):
    print(f"Tool get_order_delivery_date called for orderId: {order_id}")
    order_id_lower = order_id.lower()
    return orderIdToDeliveryDateMap.get(order_id_lower, "Unknown")

In [101]:
print(get_order_delivery_date("order1"))

Tool get_order_delivery_date called for orderId: order1
February 16, 2024


In [102]:
# There's a particular dictionary structure that's required to describe our function:

get_order_id_fn = {
    "name": "get_order_for",
    "description": "Gets the orderId given a username and order date . Call this whenever you need to know the order id for a customer given their order date, for example when a customer asks 'Hi I am Karishma, I placed an order on 21 January 2024, please provide my orderId '",
    "parameters": {
        "type": "object",
        "properties": {
            "user_name": {
                "type": "string",
                "description": "The name of the customer",
            },
            "order_date": {
                "type": "string",
                "description": "The date the order was placed in month dd, yyyy format"
            }
        },
        "required": ["user_name", "order_date"],
        "additionalProperties": False
    }
}

get_order_status_fn = {
    "name": "get_order_status",
    "description": "Gets the item status given an order id. Call this whenever you need to know the status of a user's order 'Please provide the status for my order with orderId orderxz'",
    "parameters": {
        "type": "object",
        "properties": {
            "order_id": {
                "type": "string",
                "description": "The order id the status is needed for",
            },
        },
        "required": ["order_id"],
        "additionalProperties": False
    }
}

# There's a particular dictionary structure that's required to describe our function:

get_order_delivery_date_fn = {
    "name": "get_order_delivery_date",
    "description": "Gets the delivery date of an 'In Transit' order given the orderId. Call this whenever you need to know the delivery date for an order currently in transit, for example when a customer asks 'When will my order with orderId xyz arrive'",
    "parameters": {
        "type": "object",
        "properties": {
            "order_id": {
                "type": "string",
                "description": "The order id the delivery date is needed for",
            },
        },
        "required": ["order_id"],
        "additionalProperties": False
    }
}

In [103]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": get_order_id_fn}, {"type": "function", "function": get_order_status_fn}, {"type": "function", "function": get_order_delivery_date_fn}]

In [37]:
# Let's wrap a call to GPT-4o-mini in a simple function

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        tools=tools
    )
    print(f"completion: {completion}")
    return completion.choices[0].message.content

In [33]:
print(message_gpt("I would like to enquire about my order with orderId: order1"))

completion: ChatCompletion(id='chatcmpl-Avdu3A0x2vWZhvpBscK0NRqh4XnQc', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_w7DbAYESSD60en5yxyv30wom', function=Function(arguments='{"order_id":"order1"}', name='get_order_status'), type='function')]))], created=1738301447, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_72ed7ab54c', usage=CompletionUsage(completion_tokens=18, prompt_tokens=307, total_tokens=325, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
None


In [38]:
print(message_gpt("My name is Dinesh, I placed an order on January 14, 2025, can you please provide my order status"))

completion: ChatCompletion(id='chatcmpl-AvdwmMOWh37PKBn4dei9MYFBSgmOc', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_gpiNVQtkLT4LJyFlcjVcfAu0', function=Function(arguments='{"user_name":"Dinesh","order_date":"January 14, 2025"}', name='get_order_for'), type='function')]))], created=1738301616, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_72ed7ab54c', usage=CompletionUsage(completion_tokens=30, prompt_tokens=320, total_tokens=350, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
None


In [39]:
print(message_gpt("My name is Dinesh. I had ordered a toaster, can you please provide my order status"))

completion: ChatCompletion(id='chatcmpl-AvdxpnMxrXvQBEi2nwPlPfjeq2zHj', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Could you please provide me with the date when you placed your order for the toaster?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1738301681, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_72ed7ab54c', usage=CompletionUsage(completion_tokens=19, prompt_tokens=313, total_tokens=332, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
Could you please provide me with the date when you placed your order for the toaster?


In [40]:
print(message_gpt("My name is Dinesh, I ordered a toaster on Jan 12 2024 and it hasn't arrived yet. My orderId is order3. Can you tell me when it will be delivered"))

completion: ChatCompletion(id='chatcmpl-AvdztyBxcJMzFndlAWvDixvkvyq7H', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_VcxFdGYWZA5tcD6YvPiyLOBz', function=Function(arguments='{"order_id":"order3"}', name='get_order_status'), type='function')]))], created=1738301809, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_72ed7ab54c', usage=CompletionUsage(completion_tokens=18, prompt_tokens=333, total_tokens=351, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
None


In [108]:
def chat(message, history):
    print(f'history in chat call: {json.dumps(history)}')
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model='gpt-4o-mini', messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        tool_response, tool_return_val = handle_tool_call(message)
        messages.append(message)
        messages.append(tool_response)
        response = openai.chat.completions.create(model='gpt-4o-mini', messages=messages)
    
    return response.choices[0].message.content

In [109]:
# We have to write that function handle_tool_call:

# Function lookup dictionary
available_functions = {
    "get_order_status": get_order_status,
    "get_order_for": get_order_for,
    "get_order_delivery_date": get_order_delivery_date
}

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    print(f'tool_call: {tool_call}')
    arguments = json.loads(tool_call.function.arguments)
    function_name = tool_call.function.name
    print(f'function name: {function_name}, arguments: {arguments}')
    function_to_invoke = available_functions.get(function_name)
    function_result = function_to_invoke(**arguments)  # Unpacking arguments dynamically
    print(f'function result: {function_result}')
    response = {
        "role": "tool",
        "content": json.dumps({"result": function_result,
                              "status": "partial_result"}),
        "tool_call_id": tool_call.id
    }
    return response, function_result

In [110]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.


history in chat call: []
tool_call: ChatCompletionMessageToolCall(id='call_bp80MmaReEstGx9DEwSjdl93', function=Function(arguments='{"user_name":"Dinesh","order_date":"January 20, 2024"}', name='get_order_for'), type='function')
function name: get_order_for, arguments: {'user_name': 'Dinesh', 'order_date': 'January 20, 2024'}
Tool get_order_for called for user: Dinesh, date: January 20, 2024
function result: order2
history in chat call: [{"role": "user", "metadata": {"title": null, "id": null, "parent_id": null, "duration": null, "status": null}, "content": "My name is Dinesh, I placed an order on January 20, 2024, can you please provide information on my order", "options": null}, {"role": "assistant", "metadata": {"title": null, "id": null, "parent_id": null, "duration": null, "status": null}, "content": "I'll check further details regarding your order. Please hold on. \n\nTo gather the necessary information, I'll make another tool call. \n\nExecuting now...", "options": null}]
tool_ca